In [3]:
import numpy as np
import pandas as pd
from SignalProcessor import SignalProcessorClass
import NoiseEvalUtil as NEUtil
import argparse
import os

In [6]:
bitrate = 64

In [4]:
Mixing_Path_48k_16bit = "/home/codecrack/Jnotebook/DAFX25_48k_16/Britpop/"

Noise_Generator_MP3_48k_16bit = SignalProcessorClass(filename="mixture.wav", foldpath=Mixing_Path_48k_16bit, TrackType = NEUtil.MixingType.File,bitdepth="PCM_16")
Referece_File_48k_16bit = Noise_Generator_MP3_48k_16bit.TestNoisedOnlyFile([0,0,0,0],"Reference.wav")
Mp3_Referece_File_48k_16bit = Noise_Generator_MP3_48k_16bit.GeneratingMP3RefFile(Referece_File_48k_16bit, 64)

Audio duration orginal is 41.85414965986394 seconds, now is the 8.0, the audio changing to the MONO
After LUFS, the mixing ouput in the RMS, Total: -9.39dB, Clipping Ratio&Cliped Num: (0.0010799319727891157, 381)


/root/anaconda3/lib/python3.11/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


### Visqol

In [7]:
##Loading the necessary Package
import argbind
import torch
from audiotools import AudioSignal
from audiotools import metrics

Noise_Generator_MP3_48k_16bit = SignalProcessorClass(filename="mixture.wav", foldpath=Mixing_Path_48k_16bit, TrackType = NEUtil.MixingType.File,bitdepth="PCM_16")
Referece_File_48k_16bit = Noise_Generator_MP3_48k_16bit.TestNoisedOnlyFile([0,0,0,0],"Reference.wav")
Mp3_Referece_File_48k_16bit = Noise_Generator_MP3_48k_16bit.GeneratingMP3RefFile(Referece_File_48k_16bit, bitrate)
AudioFold_48k_16bit = Mixing_Path_48k_16bit + '/Mixing_Result_Mp3_Wav/'
Reference_48k_16bit = AudioSignal(Referece_File_48k_16bit)
Mp3_Referece_48k_16bit = AudioSignal(Mp3_Referece_File_48k_16bit)

data_point_alterHN_visqol_48k_16bit = []
data_point_alterHN_visqol_mp3_48k_16bit = []
for x in range(1,100):
    filename = f"Audio_Mixing_HN{str(x)}.wav"
    HN_Gener_Audio = Noise_Generator_MP3_48k_16bit.TestNoisedOnlyFile([x,0,0,0],filename)
    HN_Gener_Audio_mp3 = Noise_Generator_MP3_48k_16bit.GeneratingMP3RefFile(HN_Gener_Audio, 64)
    HN_Gener_Audio_vis = AudioSignal(HN_Gener_Audio_mp3)  
    Visqol_score = metrics.quality.visqol(Reference_48k_16bit, HN_Gener_Audio_vis).numpy()
    mp3_Visqol_score = metrics.quality.visqol(Mp3_Referece_48k_16bit, HN_Gener_Audio_vis).numpy()
    ODG_score = round(float(Visqol_score-5),2)
    mp3_ODG_score = round(float(mp3_Visqol_score-5),2) 
    print(f"HumNoise is {x}dB. The Visqol value is {ODG_score}")
    print(f"Hum is {x}dB. The Visqol_mp3 value is {mp3_ODG_score}")
    data_point_alterHN_visqol_48k_16bit.append(ODG_score)
    data_point_alterHN_visqol_mp3_48k_16bit.append(mp3_ODG_score)

data_point_alterWN_visqol_48k_16bit = []
data_point_alterWN_visqol_mp3_48k_16bit = []
for x in range(1,100):
    filename = f"Audio_Mixing_WN{str(x)}.wav"
    WN_Gener_Audio = Noise_Generator_MP3_48k_16bit.TestNoisedOnlyFile([0,x,0,0],filename)
    WN_Gener_Audio_mp3 = Noise_Generator_MP3_48k_16bit.GeneratingMP3RefFile(WN_Gener_Audio, 64)
    WN_Gener_Audio_vis = AudioSignal(WN_Gener_Audio_mp3) 
#    WN_Gener_Audio_vis = AudioSignal(AudioFold_48k_16bit+filename)
    Visqol_score = metrics.quality.visqol(Reference_48k_16bit, WN_Gener_Audio_vis).numpy()
    mp3_Visqol_score = metrics.quality.visqol(Mp3_Referece_48k_16bit, WN_Gener_Audio_vis).numpy()
    ODG_score = round(float(Visqol_score-5),2)
    mp3_ODG_score = round(float(mp3_Visqol_score-5),2) 
    print(f"WhiteNoise is {x}dB. The Visqol value is {ODG_score}")
    data_point_alterWN_visqol_48k_16bit.append(ODG_score)
    print(f"WhiteNoise is {x}dB. The Visqol value is {mp3_ODG_score}")
    data_point_alterWN_visqol_mp3_48k_16bit.append(mp3_ODG_score)


data_point_alterCP_visqol_48k_16bit = []
data_point_alterCP_visqol_mp3_48k_16bit = []
for x in range(1,100):
    filename = f"Audio_Mixing_CP{str(x)}.wav"
    CP_Gener_Audio = Noise_Generator_MP3_48k_16bit.TestNoisedOnlyFile([0,0,x,0],filename)
    CP_Gener_Audio_mp3 = Noise_Generator_MP3_48k_16bit.GeneratingMP3RefFile(CP_Gener_Audio, 64)
    CP_Gener_Audio_vis = AudioSignal(CP_Gener_Audio_mp3)
#    CP_Gener_Audio_vis = AudioSignal(AudioFold_48k_16bit+filename)
    Visqol_score = metrics.quality.visqol(Reference_48k_16bit, CP_Gener_Audio_vis).numpy()
    ODG_score = round(float(Visqol_score- 5),2) 
    print(f"Clipping percentage is {x}%. The Visqol value is {ODG_score}")
    data_point_alterCP_visqol_48k_16bit.append(ODG_score)
    mp3_Visqol_score = metrics.quality.visqol(Mp3_Referece_48k_16bit, CP_Gener_Audio_vis).numpy()
    mp3_ODG_score = round(float(mp3_Visqol_score- 5),2) 
    print(f"Clipping percentage is {x}%. The Visqol value is {ODG_score}")
    data_point_alterCP_visqol_mp3_48k_16bit.append(mp3_ODG_score)


data_point_alterDO_visqol_48k_16bit = []
data_point_alterDO_visqol_mp3_48k_16bit = []
for x in range(1,100):
    filename = f"Audio_Mixing_DO{str(x)}.wav"
    DO_Gener_Audio = Noise_Generator_MP3_48k_16bit.TestNoisedOnlyFile([0,0,0,x],filename)
    DO_Gener_Audio_mp3 = Noise_Generator_MP3_48k_16bit.GeneratingMP3RefFile(DO_Gener_Audio, 64)
    DO_Gener_Audio_vis = AudioSignal(DO_Gener_Audio_mp3)    
#    DO_Gener_Audio_vis = AudioSignal(AudioFold_48k_16bit+filename)
    Visqol_score = metrics.quality.visqol(Reference_48k_16bit, DO_Gener_Audio_vis).numpy()
    ODG_score = round(float(Visqol_score- 5),2)
    print(f"Dropout Num is {x}. The Visqol value is {ODG_score}")
#    AK_R_score = round(CaculateAK_roughness(fileUrl),2)
    data_point_alterDO_visqol_48k_16bit.append(ODG_score)
    mp3_Visqol_score = metrics.quality.visqol(Mp3_Referece_48k_16bit, DO_Gener_Audio_vis).numpy()
    mp3_ODG_score = round(float(mp3_Visqol_score- 5),2)
    print(f"Dropout Num is {x}. The Visqol value is {mp3_Visqol_score}")
#    AK_R_score = round(CaculateAK_roughness(fileUrl),2)
    data_point_alterDO_visqol_mp3_48k_16bit.append(mp3_ODG_score)


df_HN_vis_48k_16bit = pd.DataFrame({'Hum SNR': np.arange(1,100,1), 'score': data_point_alterHN_visqol_48k_16bit})
df_HN_vis_48k_16bit = df_HN_vis_48k_16bit.apply(pd.to_numeric, errors='coerce')
df_WN_vis_48k_16bit = pd.DataFrame({'WhiteNoise SNR': np.arange(1,100,1), 'score': data_point_alterWN_visqol_48k_16bit})
df_WN_vis_48k_16bit = df_WN_vis_48k_16bit.apply(pd.to_numeric, errors='coerce')
df_CP_vis_48k_16bit = pd.DataFrame({'Clipping Percentage(%)': np.arange(1,100,1), 'score': data_point_alterCP_visqol_48k_16bit})
df_CP_vis_48k_16bit = df_CP_vis_48k_16bit.apply(pd.to_numeric, errors='coerce')
df_DO_vis_48k_16bit = pd.DataFrame({'Dropout Num': np.arange(1,100,1), 'score': data_point_alterDO_visqol_48k_16bit})
df_DO_vis_48k_16bit = df_DO_vis_48k_16bit.apply(pd.to_numeric, errors='coerce')
df_HN_vis_mp3_48k_16bit = pd.DataFrame({'Hum SNR': np.arange(1,100,1), 'score': data_point_alterHN_visqol_mp3_48k_16bit})
df_HN_vis_mp3_48k_16bit = df_HN_vis_mp3_48k_16bit.apply(pd.to_numeric, errors='coerce')
df_WN_vis_mp3_48k_16bit = pd.DataFrame({'WhiteNoise SNR': np.arange(1,100,1), 'score': data_point_alterWN_visqol_mp3_48k_16bit})
df_WN_vis_mp3_48k_16bit = df_WN_vis_mp3_48k_16bit.apply(pd.to_numeric, errors='coerce')
df_CP_vis_mp3_48k_16bit = pd.DataFrame({'Clipping Percentage(%)': np.arange(1,100,1), 'score': data_point_alterCP_visqol_mp3_48k_16bit})
df_CP_vis_mp3_48k_16bit = df_CP_vis_mp3_48k_16bit.apply(pd.to_numeric, errors='coerce')
df_DO_vis_mp3_48k_16bit = pd.DataFrame({'Dropout Num': np.arange(1,100,1), 'score': data_point_alterDO_visqol_mp3_48k_16bit})
df_DO_vis_mp3_48k_16bit = df_DO_vis_mp3_48k_16bit.apply(pd.to_numeric, errors='coerce')
# Combine the dataframes vertically (stacked on top of each other)
combined_df = pd.concat([df_HN_vis_48k_16bit, df_WN_vis_48k_16bit, df_CP_vis_48k_16bit, df_DO_vis_48k_16bit, df_HN_vis_mp3_48k_16bit, df_WN_vis_mp3_48k_16bit,df_CP_vis_mp3_48k_16bit,df_DO_vis_mp3_48k_16bit], axis=1)
# Output to CSV
##First Four Columns is vs Wav, rest four columns is vs mp3
combined_df.to_csv(f'{Mixing_Path_48k_16bit}/visqol_ind.csv', index=False)

Audio duration orginal is 41.85414965986394 seconds, now is the 8.0, the audio changing to the MONO
After LUFS, the mixing ouput in the RMS, Total: -9.39dB, Clipping Ratio&Cliped Num: (0.0010799319727891157, 381)
After LUFS, the mixing ouput in the RMS, Total: -9.0dB, Clipping Ratio&Cliped Num: (0.0004138321995464853, 146)


/root/anaconda3/lib/python3.11/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
/root/anaconda3/lib/python3.11/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


HumNoise is 1dB. The Visqol value is -0.99
Hum is 1dB. The Visqol_mp3 value is -0.55
After LUFS, the mixing ouput in the RMS, Total: -9.05dB, Clipping Ratio&Cliped Num: (0.0005158730158730159, 182)


/root/anaconda3/lib/python3.11/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


HumNoise is 2dB. The Visqol value is -0.97
Hum is 2dB. The Visqol_mp3 value is -0.52
After LUFS, the mixing ouput in the RMS, Total: -9.11dB, Clipping Ratio&Cliped Num: (0.0006065759637188208, 214)


...

Dropout Num is 97. The Visqol value is -0.89
Dropout Num is 97. The Visqol value is -0.89
There are 11025 packages
After LUFS, the mixing ouput in the RMS, Total: -9.4dB, Clipping Ratio&Cliped Num: (0.0010997732426303855, 388)


/root/anaconda3/lib/python3.11/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


Dropout Num is 98. The Visqol value is -0.93
Dropout Num is 98. The Visqol value is -0.93
There are 11025 packages
After LUFS, the mixing ouput in the RMS, Total: -9.41dB, Clipping Ratio&Cliped Num: (0.0010487528344671202, 370)


/root/anaconda3/lib/python3.11/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


Dropout Num is 99. The Visqol value is -0.95
Dropout Num is 99. The Visqol value is -0.95


### HAAQI

In [8]:
import clarity
import clarity.evaluator.haaqi as haaqi
import librosa

In [9]:
from clarity.utils.audiogram import Audiogram
levels_1 = np.array([0, 0, 0, 0, 0, 0, 0, 0])
audiogram_NH = Audiogram(levels=levels_1)

In [10]:
Noise_Generator_MP3_48k_16bit = SignalProcessorClass(filename="mixture.wav", foldpath=Mixing_Path_48k_16bit, TrackType = NEUtil.MixingType.File,bitdepth="PCM_16")
Referece_File_48k_16bit = Noise_Generator_MP3_48k_16bit.TestNoisedOnlyFile([0,0,0,0],"Reference.wav")
Mp3_Referece_File_48k_16bit = Noise_Generator_MP3_48k_16bit.GeneratingMP3RefFile(Referece_File_48k_16bit, bitrate)

AudioFold_48k_16bit = f'{Mixing_Path_48k_16bit}/Mixing_Result_Mp3_Wav/'
Mp3_Referece_data_48k_16bit, sample_rate = librosa.load(Mp3_Referece_File_48k_16bit, sr=None) 
Referece_data_48k_16bit, sample_rate = librosa.load(Referece_File_48k_16bit, sr=None)

while (os.path.isfile(Referece_File_48k_16bit) and os.path.isfile(Mp3_Referece_File_48k_16bit)):
    print ("file now ready")
    break

data_point_alterHN_HAAQI_48k_16bit = []
data_point_alterHN_HAAQI_mp3_48k_16bit = []    
for x in range(1,100):
    filename = f"Audio_Mixing_HN{str(x)}.wav"
    HN_Gener_Audio = Noise_Generator_MP3_48k_16bit.TestNoisedOnlyFile([x,0,0,0],filename)
    HN_Gener_Audio_mp3 = Noise_Generator_MP3_48k_16bit.GeneratingMP3RefFile(HN_Gener_Audio, 64)
    HN_Gener_Audio_HAAQI, sample_rate = librosa.load(HN_Gener_Audio_mp3, sr=None)
#    HN_Gener_Audio_HAAQI, sample_rate = librosa.load(AudioFold_48k_16bit+filename, sr=None)
    HAAQI_score = round(haaqi.compute_haaqi(HN_Gener_Audio_HAAQI,Referece_data_48k_16bit,sample_rate,sample_rate,audiogram_NH),2)
#    ODG_score = round(float(Visqol_score-5),2) 
    print(f"HumNoise is {x}dB. The HAAQI value is {HAAQI_score}")
    data_point_alterHN_HAAQI_48k_16bit.append(HAAQI_score)
    mp3_HAAQI_score = round(haaqi.compute_haaqi(HN_Gener_Audio_HAAQI,Mp3_Referece_data_48k_16bit,sample_rate,sample_rate,audiogram_NH),2)
    print(f"HumNoise is {x}dB. The HAAQI value is {mp3_HAAQI_score}")
    data_point_alterHN_HAAQI_mp3_48k_16bit.append(mp3_HAAQI_score)

data_point_alterWN_HAAQI_48k_16bit = []
data_point_alterWN_HAAQI_mp3_48k_16bit = []
for x in range(1,100):
    filename = f"Audio_Mixing_WN{str(x)}.wav"
    WN_Gener_Audio = Noise_Generator_MP3_48k_16bit.TestNoisedOnlyFile([0,x,0,0],filename)
    WN_Gener_Audio_mp3 = Noise_Generator_MP3_48k_16bit.GeneratingMP3RefFile(WN_Gener_Audio, 64)
    WN_Gener_Audio_HAAQI, sample_rate = librosa.load(WN_Gener_Audio_mp3, sr=None)
    HAAQI_score = round(haaqi.compute_haaqi(WN_Gener_Audio_HAAQI,Referece_data_48k_16bit,sample_rate,sample_rate,audiogram_NH),2)
#    ODG_score = round(float(Visqol_score-5),2) 
    print(f"WhiteNoise is {x}dB. The HAAQI value is {HAAQI_score}")
    data_point_alterWN_HAAQI_48k_16bit.append(HAAQI_score)
    mp3_HAAQI_score = round(haaqi.compute_haaqi(WN_Gener_Audio_HAAQI,Mp3_Referece_data_48k_16bit,sample_rate,sample_rate,audiogram_NH),2)
    print(f"WhiteNoise is {x}dB. The HAAQI value is {mp3_HAAQI_score}")
    data_point_alterWN_HAAQI_mp3_48k_16bit.append(mp3_HAAQI_score)

data_point_alterCP_HAAQI_48k_16bit = []
data_point_alterCP_HAAQI_mp3_48k_16bit = []
for x in range(1,100):
    filename = f"Audio_Mixing_CP{str(x)}.wav"
    CP_Gener_Audio = Noise_Generator_MP3_48k_16bit.TestNoisedOnlyFile([0,0,x,0],filename)
    CP_Gener_Audio_mp3 = Noise_Generator_MP3_48k_16bit.GeneratingMP3RefFile(CP_Gener_Audio, 64)    
    CP_Gener_Audio_HAAQI, sample_rate = librosa.load(CP_Gener_Audio_mp3, sr=None)
    HAAQI_score = round(haaqi.compute_haaqi(CP_Gener_Audio_HAAQI,Referece_data_48k_16bit,sample_rate,sample_rate,audiogram_NH),2)
    print(f"Clipping percentage is {x}%. The HAAQI value is {HAAQI_score}")
    data_point_alterCP_HAAQI_48k_16bit.append(HAAQI_score)
    mp3_HAAQI_score = round(haaqi.compute_haaqi(CP_Gener_Audio_HAAQI,Mp3_Referece_data_48k_16bit,sample_rate,sample_rate,audiogram_NH),2)
    print(f"Clipping percentage is {x}%. The HAAQI value is {mp3_HAAQI_score}")
    data_point_alterCP_HAAQI_mp3_48k_16bit.append(mp3_HAAQI_score)

data_point_alterDO_HAAQI_48k_16bit = []
data_point_alterDO_HAAQI_mp3_48k_16bit = []
for x in range(1,100):
    filename = f"Audio_Mixing_DO{str(x)}.wav"
    DO_Gener_Audio = Noise_Generator_MP3_48k_16bit.TestNoisedOnlyFile([0,0,0,x],filename)
    DO_Gener_Audio_mp3 = Noise_Generator_MP3_48k_16bit.GeneratingMP3RefFile(DO_Gener_Audio, 64)      
    DO_Gener_Audio_HAAQI, sample_rate = librosa.load(DO_Gener_Audio_mp3, sr=None)
    HAAQI_score = round(haaqi.compute_haaqi(DO_Gener_Audio_HAAQI,Referece_data_48k_16bit,sample_rate,sample_rate,audiogram_NH),2)
    print(f"Dropout Num is {x}. The HAAQI value is {HAAQI_score}")
    data_point_alterDO_HAAQI_48k_16bit.append(HAAQI_score)
    mp3_HAAQI_score = round(haaqi.compute_haaqi(DO_Gener_Audio_HAAQI,Mp3_Referece_data_48k_16bit,sample_rate,sample_rate,audiogram_NH),2)
    print(f"Dropout Num is {x}. The HAAQI value is {mp3_HAAQI_score}")
    data_point_alterDO_HAAQI_mp3_48k_16bit.append(mp3_HAAQI_score)


df_HN_HAAQI_48k_16bit = pd.DataFrame({'HumNoise SNR': np.arange(1,100,1), 'score': data_point_alterHN_HAAQI_48k_16bit})
df_HN_HAAQI_48k_16bit = df_HN_HAAQI_48k_16bit.apply(pd.to_numeric, errors='coerce')
df_WN_HAAQI_48k_16bit = pd.DataFrame({'WhiteNoise SNR': np.arange(1,100,1), 'score': data_point_alterWN_HAAQI_48k_16bit})
df_WN_HAAQI_48k_16bit = df_WN_HAAQI_48k_16bit.apply(pd.to_numeric, errors='coerce')
df_CP_HAAQI_48k_16bit = pd.DataFrame({'Clipping Percentage(%)': np.arange(1,100,1), 'score': data_point_alterCP_HAAQI_48k_16bit})
df_CP_HAAQI_48k_16bit = df_CP_HAAQI_48k_16bit.apply(pd.to_numeric, errors='coerce')
df_DO_HAAQI_48k_16bit = pd.DataFrame({'Dropout Num': np.arange(1,100,1), 'score': data_point_alterDO_HAAQI_48k_16bit})
df_DO_HAAQI_48k_16bit = df_DO_HAAQI_48k_16bit.apply(pd.to_numeric, errors='coerce')
df_HN_HAAQI_mp3_48k_16bit = pd.DataFrame({'HumNoise SNR': np.arange(1,100,1), 'score': data_point_alterHN_HAAQI_mp3_48k_16bit})
df_HN_HAAQI_mp3_48k_16bit = df_HN_HAAQI_mp3_48k_16bit.apply(pd.to_numeric, errors='coerce')
df_WN_HAAQI_mp3_48k_16bit= pd.DataFrame({'WhiteNoise SNR': np.arange(1,100,1), 'score': data_point_alterWN_HAAQI_mp3_48k_16bit})
df_WN_HAAQI_mp3_48k_16bit = df_WN_HAAQI_mp3_48k_16bit.apply(pd.to_numeric, errors='coerce')
df_CP_HAAQI_mp3_48k_16bit = pd.DataFrame({'Clipping Percentage(%)': np.arange(1,100,1), 'score': data_point_alterCP_HAAQI_mp3_48k_16bit})
df_CP_HAAQI_mp3_48k_16bit = df_CP_HAAQI_mp3_48k_16bit.apply(pd.to_numeric, errors='coerce')
df_DO_HAAQI_mp3_48k_16bit = pd.DataFrame({'Dropout Num': np.arange(1,100,1), 'score': data_point_alterDO_HAAQI_mp3_48k_16bit})
df_DO_HAAQI_mp3_48k_16bit = df_DO_HAAQI_mp3_48k_16bit.apply(pd.to_numeric, errors='coerce')

# Combine the dataframes vertically (stacked on top of each other)
combined_df = pd.concat([df_HN_HAAQI_48k_16bit, df_WN_HAAQI_48k_16bit, df_CP_HAAQI_48k_16bit, df_DO_HAAQI_48k_16bit, df_HN_HAAQI_mp3_48k_16bit, df_WN_HAAQI_mp3_48k_16bit,df_CP_HAAQI_mp3_48k_16bit,df_DO_HAAQI_mp3_48k_16bit], axis=1)
# Output to CSV
##First Four Columns is vs Wav, rest four columns is vs mp3
combined_df.to_csv(f'{Mixing_Path_48k_16bit}/haaqi_ind.csv', index=False)


Audio duration orginal is 41.85414965986394 seconds, now is the 8.0, the audio changing to the MONO
After LUFS, the mixing ouput in the RMS, Total: -9.39dB, Clipping Ratio&Cliped Num: (0.0010799319727891157, 381)
file now ready
After LUFS, the mixing ouput in the RMS, Total: -9.0dB, Clipping Ratio&Cliped Num: (0.0004138321995464853, 146)


/root/anaconda3/lib/python3.11/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")
/root/anaconda3/lib/python3.11/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


HumNoise is 1dB. The HAAQI value is 0.18
HumNoise is 1dB. The HAAQI value is 0.55
After LUFS, the mixing ouput in the RMS, Total: -9.05dB, Clipping Ratio&Cliped Num: (0.0005158730158730159, 182)


/root/anaconda3/lib/python3.11/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


HumNoise is 2dB. The HAAQI value is 0.19
HumNoise is 2dB. The HAAQI value is 0.57
After LUFS, the mixing ouput in the RMS, Total: -9.11dB, Clipping Ratio&Cliped Num: (0.0006065759637188208, 214)


...

Dropout Num is 97. The HAAQI value is 0.2
Dropout Num is 97. The HAAQI value is 0.76
There are 11025 packages
After LUFS, the mixing ouput in the RMS, Total: -9.4dB, Clipping Ratio&Cliped Num: (0.0010997732426303855, 388)


/root/anaconda3/lib/python3.11/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


Dropout Num is 98. The HAAQI value is 0.2
Dropout Num is 98. The HAAQI value is 0.71
There are 11025 packages
After LUFS, the mixing ouput in the RMS, Total: -9.42dB, Clipping Ratio&Cliped Num: (0.0010884353741496598, 384)


/root/anaconda3/lib/python3.11/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


Dropout Num is 99. The HAAQI value is 0.2
Dropout Num is 99. The HAAQI value is 0.73


In [14]:
HN_Gener_Audio_mp3

'/home/codecrack/Jnotebook/DAFX25_48k_16/Grunge/Mixing_Result_Mp3_Wav/Audio_Mixing_HN99_64kbps_64kbps.wav'

In [12]:
Referece_File_48k_16bit

'/home/codecrack/Jnotebook/DAFX25_48k_16/Grunge//Mixing_Result/Reference.wav'